In [4]:
install.packages(c("doParallel", "foreach"))

Installing packages into ‘/home/jupyter/packages’
(as ‘lib’ is unspecified)

also installing the dependency ‘iterators’




In [1]:
library(foreach)
library(doParallel)

Loading required package: iterators

Loading required package: parallel



In [1]:
start_time = Sys.time()

In [3]:
threads = 16
cl = makeCluster(threads)
registerDoParallel(cl)

options(scipen=999)

chr = 22 

window_eagle = 5e6
window_minimac = 500000
chunk_size = 20e4

target_vcf = paste0("chr", chr, "/arrays_QCed-updated-chr", chr, ".vcf.gz")
bim = paste0("chr", chr, "/arrays_QCed-updated-chr",chr,".bim")
bcf_ref = paste0("ref/eur_bcfs/ALL.chr", chr, ".phase3_v5.shapeit2_mvncall_integrated.noSingleton.genotypes.recode.bcf")
gen_map = "ref/map/genetic_map_hg19_withX.txt.gz"
m3vcf = paste0("ref/m3vcfs/", chr, ".1000g.Phase3.v5.With.Parameter.Estimates.msav")
id_file = "ref/EUR.sample"

# Split chunk

In [4]:
a = read.table(bim)

bp = seq(min(a$V4), max(a$V4), by=chunk_size)

if (tail(bp, 1) != max(a$V4)) {
    bp = c(bp, max(a$V4))
}


skip_start = FALSE
start_list = NULL
end_list = NULL

for (i in 1:(length(bp)-1)) {

    if (!skip_start) {
        start = bp[i]
    }

    if (i == (length(bp)-1)) {
        end = bp[i+1]
    } else {
        end = bp[i+1] -1
    }

    n_snp = sum(a$V4 >= start & a$V4 <= end)

    if (n_snp < 3 & i != (length(bp)-1)) {
        skip_start = TRUE
        next
    } else if (n_snp < 3 & i == (length(bp)-1)) {
        end_list[length(end_list)] = end
    } else {
        skip_start = FALSE
        start_list = c(start_list, start)
        end_list = c(end_list, end)
    }
}

print(length(start_list))

[1] 169


# Phasing and Imputation

In [5]:
res = foreach(i=1:length(start_list)) %dopar% {
    start = start_list[i]
    end = end_list[i]
    prefix = paste0("chr", chr, "/chunk_", i, "_", start, "_", end)

    # subset to chunk
    system(paste0("bcftools view -r ", chr, ":", start, "-", end, " ", target_vcf, " -Oz -o ", prefix, ".vcf.gz"))
    system(paste0("tabix ", prefix, ".vcf.gz"))

    # phasing
    system(paste0("Eagle_v2.4.1/eagle --pbwtIters 1 --chrom ", chr, " --vcfRef ", bcf_ref, " --vcfTarget ",  prefix, ".vcf.gz --geneticMapFile ", gen_map, " --outPrefix ", prefix, ".phased --bpStart ", start, " --bpEnd ", end, " --bpFlanking ", window_eagle, " --allowRefAltSwap --vcfOutFormat z 2> chr", chr, "/err", i, ".log 1> chr", chr, "/out", i, ".log"))
    system(paste0("tabix ", prefix, ".phased.vcf.gz"))

    # imputation
    system(paste0("Minimac4/bin/minimac4 --threads 1 --min-r2 0.3 --overlap 500000 --format GT,DS,GP --all-typed-sites -O vcf.gz --min-ratio 0.00001 --sample-ids-file ", id_file, " --region ", chr, ":", start, "-", end, " -o ", prefix, ".dose.vcf.gz ", m3vcf, " ", prefix, ".phased.vcf.gz 2>> chr", chr, "/out", i, ".log"))
    system(paste0("tabix ", prefix, ".dose.vcf.gz"))

    # clean up
    system(paste0("rm -f ", prefix, ".vcf.gz*"))
    system(paste0("rm -f ", prefix, ".phased.vcf.gz*"))
    system(paste0("rm -f chr", chr, "/*", i, ".log"))
}

stopCluster(cl)

# Merge all files

In [27]:
files = list.files(paste0("chr", chr), "*.dose.vcf.gz$", full.names=T)
files = sort(files)
system(paste0("bcftools concat ", paste0(files, collapse=" "), " -O z -o chr", chr, "/chr", chr, ".vcf.gz --threads ", threads))
system(paste0("tabix chr ", chr, "/chr", chr, ".vcf.gz"))
system(paste0("rm -f chr", chr, "/*.dose.vcf.gz*"))

In [2]:
end_time = Sys.time()

In [3]:
print(end_time - start_time)

Time difference of 9.920631 secs
